![image](https://tech.gmogshd.com/wp-content/uploads/2023/03/%E3%82%B9%E3%82%AF%E3%83%AA%E3%83%BC%E3%83%B3%E3%82%B7%E3%83%A7%E3%83%83%E3%83%88-2023-03-17-0.06.54.png)

# transfomer

## Feature

- encoderとdecoderが積み重なっている
- 文章を翻訳時encoderで文章を入力
- decoderで過去の文章を繰り返し学習する。encoderで入力があった時学習したモデルと照らし合わせて一番似ている文字を出力する

## Implementation
- tensorflowのtutorialを通してattentionを実装する

## ⚠️Note
- P100GPU1基で実行しても10分ほどかかる


In [2]:
import tensorflow as tf

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split

import unicodedata
import re
import numpy as np
import os
import io
import time


データセットをダウンロードしたあと、データを準備するために下記のようないくつかの手順を実行します。

- それぞれの文ごとに、開始 と 終了 のトークンを付加する
- 特殊文字を除去して文をきれいにする
- 単語インデックスと逆単語インデックス（単語 → id と id → 単語のマッピングを行うディクショナリ）を作成する
- 最大長にあわせて各文をパディングする


In [3]:
# path_to_zip = tf.keras.utils.get_file(
#     fname='spa-eng.zip', 
#     origin='http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip',
#     extract=True
#     )

path_to_file = "../../data/spa-eng/spa.txt"


In [4]:
def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')

def preprocess_sentence(w):
    w = unicode_to_ascii(w.lower().strip())

    # 単語とその後の句読点の間にスペースを挿入
    # e.g. "he is a boy." => "he is a boy ."
    # 参照：- https://stackoverflow.com/questions/3645931/python-padding-punctuation-with-white-spaces-keeping-punctuation
    w = re.sub(r"([?.!,¿])", r" \1 ", w)
    w = re.sub(r'[" "]', " ", w)

    # (a-z, A-Z, ".", "?", "!", ".")以外のすべての記号をスペースに置き換え
    w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)

    w = w.rstrip().strip()

    # 文の開始と終了のトークンを付与
    # モデルが予測をいつ開始し、終了すればいいかを知らせるため
    w = '<start> ' + w + ' <end>'
    return w


In [5]:
# 事前処理を見てみる
en_sentence = u"May I borrow this book?"
sp_sentence = u"¿Puedo tomar prestado este libro?"
print(preprocess_sentence(en_sentence))
print(preprocess_sentence(sp_sentence).encode('utf-8'))

# print(b"\xc2\xbf".decode('utf-8'))


<start> may i borrow this book ? <end>
b'<start> \xc2\xbf puedo tomar prestado este libro ? <end>'


In [6]:
def create_dataset(path, num_examples):
    """
    firstly, create_dataset removes accent sign, then clean the sentence
    return: pair of [ENGLISH, SPANISH]
    """
    lines = io.open(path, encoding='UTF-8').read().strip().split("\n")
    word_pairs = [[preprocess_sentence(w) for w in l.split("\t")] for l in lines[:num_examples]]

    return zip(*word_pairs)


In [7]:
en, sp = create_dataset(path_to_file, None)
print(en[-1])
print(sp[-1])


<start> if you want to sound like a native speaker , you must be willing to practice saying the same sentence over and over in the same way that banjo players practice the same phrase over and over until they can play it correctly and at the desired tempo . <end>
<start> si quieres sonar como un hablante nativo , debes estar dispuesto a practicar diciendo la misma frase una y otra vez de la misma manera en que un musico de banjo practica el mismo fraseo una y otra vez hasta que lo puedan tocar correctamente y en el tiempo esperado . <end>


In [8]:
def max_length(tensor):
    return max(len(t) for t in tensor)


In [9]:
def tokenize(lang):
    lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(
        filters=''
    )
    lang_tokenizer.fit_on_texts(lang)

    tensor = lang_tokenizer.text_to_sequence(lang)
    tensor = tf.keras.preprocessing.sequence.pad_sequences(
        tensor,
        padding='post'
    )

    return tensor, lang_tokenizer


In [10]:
def load_dataset(path, num_examples=None):
    targ_lang, inp_lang = create_dataset(path, num_examples)

    input_tensor, inp_lang_tokenizer = tokenize(inp_lang)
    target_tensor, targ_lang_tokenizer = tokenize(targ_lang)

    return input_tensor, target_tensor, inp_lang_tokenizer, targ_lang_tokenizer


実験を早くするためにデータセットのサイズを制限

In [11]:
# このサイズのデータセットで実験
num_examples = 30000
input_tensor, target_tensor, inp_lang, targ_lang = load_dataset(path_to_file, num_examples)

# ターゲットテンソルの最大長を計算
max_length_targ, max_length_inp = max_length(target_tensor), max_length(input_tensor)


AttributeError: 'Tokenizer' object has no attribute 'text_to_sequence'

In [12]:
# 80-20で分割を行い、訓練用と検証用のデータセットを作成
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.2)

# 長さを表示
print(len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val))


NameError: name 'input_tensor' is not defined

In [13]:
def convert(lang, tensor):
  for t in tensor:
    if t!=0:
      print ("%d ----> %s" % (t, lang.index_word[t]))


In [ ]:
print ("Input Language; index to word mapping")
convert(inp_lang, input_tensor_train[0])
print ()
print ("Target Language; index to word mapping")
convert(targ_lang, target_tensor_train[0])


tf.dataデータセットの作成

In [ ]:
BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 64
steps_per_epoch = len(input_tensor_train)//BATCH_SIZE
embedding_dim = 256
units = 1024
vocab_inp_size = len(inp_lang.word_index)+1
vocab_tar_size = len(targ_lang.word_index)+1

dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)


In [ ]:
example_input_batch, example_target_batch = next(iter(dataset))
example_input_batch.shape, example_target_batch.shape


TODO
- エンコーダー, デコーダーを記述
- 翻訳を試す